# Problem Set 3: Merging and regular expressions

**Total points (without extra credit)**: 44

**Background on the policy context**: here, we're going to use two datasets to practice reshaping, merging, and regular expression patterns. Both datasets relate to the broader issue of which employers might be violating the rights of temporary guestworkers granted visas under the H-2A program. Here are some articles about potential exploitation of guestworkers by firms and inequality caused by minimal oversight:

- News media coverage of labor abuses of temporary guestworkers: https://www.buzzfeednews.com/article/kenbensinger/the-pushovers 
- GAO report on labor abuses of temporary guestworkers: https://www.gao.gov/products/gao-15-154

The following datasets are located in `pset3_inputdata` (need to unzip): 

- `jobs`: a dataset of guestworker jobs posted by many employers, some of whom have been debarred (banned) from the program for labor abuses; others not debarred
- `debar`: a dataset of employers who committed violations of labor regulations meant to protect temporary guestworkers 


## Resources from class

- [Lecture](https://github.com/jhaber-zz/QSS20_public/blob/main/slides/06_qss20_w23_mergereshape.pdf) and [activity](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/03_reshaping_merging_solutions.ipynb) on exact merging
- [Lecture](https://github.com/jhaber-zz/QSS20_public/blob/main/slides/07_qss20_w23_regex.pdf) and [activity](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/04_regex_blank.ipynb) on regular expressions (both coming in class on Feb. 1)
- DataCamp modules on both of these

# 0. Load packages & data (1 point total)

In [1]:
## helpful packages
import pandas as pd
import numpy as np
import random
import re
import os

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


## 0.1 Load data (0 points)

Load the following dataset stored in `pset3_inputdata`&mdash;`debar.csv`&mdash;storing it as a dataframe named `debar`. This represents employers temporarily banned from hiring workers.

View the `head()` and columns of this dataframe.

In [2]:
## your code here
debar = pd.read_csv('pset3_inputdata/debar.csv')
debar.head()
debar.columns

debar['City, State']

,Name,"City, State",Violation,Duration,Start date,End date
0,J&J Harvesting,"Leads, ND",Failure to respond to audit (partial response),2 years,1/19/2014,1/18/2016
1,"Stahlman Apiaries, Inc","Selby, SD",Failure to respond to audit (partial response),1 year,2/19/2015,2/14/2016
2,Trust Nursery,"Pulaski, NY",Failure to respond to audit (partial response),1 year,3/21/2014,3/20/2015
3,Anton Fertilizer Inc.,"Dighton, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016
4,"Great Plains Fluid Service, Inc.","Greensburg, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016


Index(['Name', 'City, State', 'Violation', 'Duration', 'Start date',
       'End date'],
      dtype='object')

0             Leads, ND
1             Selby, SD
2           Pulaski, NY
3           Dighton, KS
4        Greensburg, KS
             ...       
109    Mount Vernon, TX
110        Mulberry, FL
111           Healy, KS
112    Collins, Georgia
113      Lyons, Georgia
Name: City, State, Length: 114, dtype: object

The most notable column names mean as follows:
- `Name`:	Company name of agricultural employer
- `City, State`:	City and state where employer located
- `Violation`:	Type of program violation
- `Start date`:	Start date of debarment (temporary ban)

## 0.2 Inspect data (1 point)

Print the number of rows in `debar` versus the number of unique employer names (`Name`). Is there one row per employer or multiple rows for some employers?

In [3]:
## your code here
debar['Name'].nunique()
debar.shape
## There are 98 unique employer name whereas the dataset has 114 entries, indicating that there are multiple rows for some employers.

98

(114, 6)

# 1. Reshape data and check duplicates (19 points total)

## 1.1 Make indicator for violation number (2 points)

To make it possible to reshape data, make an indicator for the violation number for each business. The indicator should take the value of `viol` if it's the first row/potential violation, `viol2` if the second row/potential violation, etc.

**Hint:**
- One way to do this is by using an if-else statement to check whether the business name is the same as in the row above (assuming rows are ordered by name). Grouping by employer name and checking the number of unique offenses would also work.

In [4]:
## your code here
debar['violation_number'] = debar.groupby('Name').cumcount() + 1
debar['violation_number'] = debar['violation_number'].apply(lambda x: 'viol' + str(x) if x > 1 else 'viol')
debar['violation_number'].value_counts()


viol     98
viol2    16
Name: violation_number, dtype: int64

## 1.2 Clean up state names (3 points)

Inspect the state names in the business violation data. Which states have names sometimes indicated in long format vs. two-letter abbreviation, e.g. "New Hampshire" vs. "NH"? Which of these may have more than one violation?

**Hint:** 
- One way to do this is to extract state names from the `City, State` column using regex and/or string methods.
- The simplest way to check if a state may have more than one violation is to check how many times they appear using `value_counts()`. 

In [97]:
## your code here
## extract state names from the 'City, State' column
debar['State'] = debar['City, State'].str.extract(r',\s+(.*)')

## long form indicator
debar['State_indicator'] = debar['State'].apply(lambda x: True if len(str(x)) > 2 else False)
#debar['State'].value_counts()
long_state_names = set(debar[(~debar.State.isna()) & (debar.State_indicator == True)]['State'].unique())

long_state_names # a set of all the states with a long-format name

{'Arkansas',
 'California',
 'Florida',
 'Georgia',
 'Idaho',
 'Kansas',
 'Louisiana',
 'Massachusetts',
 'North Carolina',
 'North Dakota',
 'Tennessee',
 'Texas',
 'Utah',
 'Vermont'}

In [98]:
## violation count of all the states in the long-format name
debar[debar.State.isin(long_state_names)]['State'].value_counts()

Georgia           9
Texas             4
North Dakota      3
Utah              2
Florida           2
Kansas            1
California        1
Arkansas          1
Vermont           1
Tennessee         1
Louisiana         1
North Carolina    1
Idaho             1
Massachusetts     1
Name: State, dtype: int64

In [102]:
## subset only the states with a violation count of more than one
long_format_count = debar[debar.State.isin(long_state_names)]['State'].value_counts()
long_format_count = long_format_count[long_format_count > 1]
long_format_count

Georgia         9
Texas           4
North Dakota    3
Utah            2
Florida         2
Name: State, dtype: int64

To make the state names consistent with the jobs data below, convert any discrepant state names to the two-letter abbreviation format. 

**Hints:**
- You could do this for all state names (to be safe) or only those you just identified. 
- If you want to change ANY discrepant state names from long format to two-letter format (i.e., a complete conversion), you can load state names from a complete name/abbreviation crosswalk and use that to change state names. Here is code to load in such a list ([from this blog](https://towardsdatascience.com/state-name-to-state-abbreviation-crosswalks-6936250976c)):
```python
crosswalk_url = 'http://app02.clerk.org/menu/ccis/Help/CCIS%20Codes/state_codes.html'
pd.read_html(crosswalk_url)[0]
```

In [107]:
## your code here
crosswalk_url = 'http://app02.clerk.org/menu/ccis/Help/CCIS%20Codes/state_codes.html'
state_crosswalk = pd.read_html(crosswalk_url)[0]
state_crosswalk.columns = ['Abbreviation', 'State']

def convert_to_abbreviation(state_name):
    """Convert state name to abbreviation"""
    abbreviation = state_crosswalk[state_crosswalk['State'] == state_name]['Abbreviation'].values
    ## return the first element of the array if the abbreviation is successful
    if len(abbreviation) > 0:
        return abbreviation[0]
    else:
        return state_name

debar['State_cleaned'] = debar['State'].apply(convert_to_abbreviation)

## print the cleaned state column and compare it to the original state column
debar[['City, State', 'State', 'State_cleaned']].sample(20)

,"City, State",State,State_cleaned
3,"Dighton, KS",KS,KS
1,"Selby, SD",SD,SD
51,"Volga, SD",SD,SD
93,"Franklin, North Carolina",North Carolina,NC
50,"Rifle, CO",CO,CO
43,"Miami, FL",FL,FL
29,"Groton, MA",MA,MA
82,"Froid, MT",MT,MT
103,"Mount Vernon, TX",TX,TX
37,"Carlisle, KY",KY,KY


In [104]:
## count the number of violations for each state on the cleaned state column
debar['State_cleaned'].value_counts()

GA    19
TX    18
KS    10
FL    10
ND     6
UT     6
SD     5
KY     5
CA     4
AR     3
NY     3
MA     3
CO     3
LA     3
MT     2
ID     2
OK     2
MN     1
SC     1
VT     1
TN     1
AK     1
MD     1
NC     1
IL     1
ME     1
Name: State_cleaned, dtype: int64

## 1.3 Investigate duplicated rows (4 points)

A. Create a new column in `debar`, `is_repeated`, that tells us whether an employer (`Name`) is repeated > 1 times.

*Hint*: there are multiple ways to solve this but some possibilities to get the list of names that are repeated are:
- Using `value_counts()` on the `Name` variable and extracting the index from that value counts 
- Using groupby to count the rows attached to one name

B. Print the rows where `is_repeated == True` and interpret with at least a sentence. If you notice any cases of duplicate business names where 'City, State' does NOT match exactly, check these manually and make them consistent.

C. Subset to the rows where `is_repeated == True` and save that data as `mult_debar`. Print the head() and shape

In [129]:
## your code here
employee_count = debar['Name'].value_counts()
repeated_employees = set(employee_count[employee_count > 1].index)

## create a boolean indicator for repeated employees
debar['is_repeated'] = debar.Name.isin(repeated_employees)

## print all the repeated employees
repeated_employees


{'Annabella Land & Cattle',
 'Autumn Hill Orchard',
 'Caddo Creek Ranch, dba Paradise Ranch',
 'Cisco Produce Inc.',
 'Dove Creek Farms',
 'F&W Farms',
 'Loewen Harvesting LLC',
 'Macky and Brad Farms',
 'Maple Ridge Custom Services, LLC',
 'Mark Duncan',
 'Old Tree Farms/Verpaalen Custom Service',
 'Rollo Farm Labor Contractor',
 'SRT Farms',
 'Sharon Mathis',
 'Turner Farms',
 'Xavier Horne'}

In [142]:
## print the rows where is_repeated is True
debar[debar.is_repeated == True].sort_values('Name')

,Name,"City, State",Violation,Duration,Start date,End date,violation_number,State,State_indicator,State_cleaned,is_repeated
6,Annabella Land & Cattle,"Annabella, UT",Non Payment,1 year,5/9/2014,5/9/2015,viol,UT,False,UT,True
30,Annabella Land & Cattle,"Annabella, Utah",Non-payment,1 year,5/9/2014,5/8/2015,viol2,Utah,True,UT,True
29,Autumn Hill Orchard,"Groton, MA",Impeding the Audit Process – Non- Response,2 years,7/6/2014,7/5/2016,viol2,MA,False,MA,True
7,Autumn Hill Orchard,"Groton, MA",Failure to respond to audit (no response),2 years,7/6/2014,7/5/2016,viol,MA,False,MA,True
8,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",Failure to respond to audit (partial response),2 years,7/20/2014,7/19/2016,viol,TX,False,TX,True
28,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, Texas",Impeding the Audit Process – Partial- Response,2 years,7/20/2014,7/19/2016,viol2,Texas,True,TX,True
56,Cisco Produce Inc.,"Cairo, GA",Impeding the Audit Process – Non- Response,2 years,12/10/2015,12/9/2017,viol2,GA,False,GA,True
19,Cisco Produce Inc.,"Cairo, GA",Failure to respond to audit (no response),2 years,12/10/2014,12/9/2016,viol,GA,False,GA,True
103,Dove Creek Farms,"Mount Vernon, TX",Failure to respond to audit request,2 years,2/9/2018,2/8/2018,viol,TX,False,TX,True
109,Dove Creek Farms,"Mount Vernon, TX",Failure to Respond to Audit Request,2 years,2/9/2018,2/8/2020,viol2,TX,False,TX,True


We noticed that for 'Maple Ridge Custom Services, LLC' the states for the two entries are not the same. It says `Altheimer, AK` for the first and `Altheimer, AK` for the second. It seems like the person made a mistake while entering the data to the database. Since, `Altheimer` is a city in Arkansas(AK), we are going to perform data cleaning manually.

In [139]:
## edit the state name manually and print the edited rows
debar.loc[debar.Name == "Maple Ridge Custom Services, LLC", ['City, State', 'State', 'State_cleaned']] = ["Altheimer, AR", "AR", "AR"]
debar.loc[debar.Name == "Maple Ridge Custom Services, LLC", ]

,Name,"City, State",Violation,Duration,Start date,End date,violation_number,State,State_indicator,State_cleaned,is_repeated
17,"Maple Ridge Custom Services, LLC","Altheimer, AR",Failure to respond to audit (partial response),2 years,11/16/2014,11/15/2016,viol,AR,False,AR,True
59,"Maple Ridge Custom Services, LLC","Altheimer, AR",Impeding the Audit Process – Partial- Response,1 year,11/16/2014,11/15/2015,viol2,AR,False,AR,True


In [143]:
# Subset to the rows where `is_repeated == True` and save that data as `mult_debar`. Print the head() and shape
mult_debar = debar[debar.is_repeated == True]
mult_debar.head()
mult_debar.shape

,Name,"City, State",Violation,Duration,Start date,End date,violation_number,State,State_indicator,State_cleaned,is_repeated
6,Annabella Land & Cattle,"Annabella, UT",Non Payment,1 year,5/9/2014,5/9/2015,viol,UT,False,UT,True
7,Autumn Hill Orchard,"Groton, MA",Failure to respond to audit (no response),2 years,7/6/2014,7/5/2016,viol,MA,False,MA,True
8,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",Failure to respond to audit (partial response),2 years,7/20/2014,7/19/2016,viol,TX,False,TX,True
11,Loewen Harvesting LLC,"Brownsville, TX",Failure to respond to audit (partial response),1 year,8/20/2014,8/19/2015,viol,TX,False,TX,True
12,Rollo Farm Labor Contractor,"Miami, FL",Failure to respond to audit (no response),2 years,8/23/2014,8/22/2016,viol,FL,False,FL,True


(32, 11)

## 1.4 Reshape `mult_debar` to wide format to begin filtering out duplicates (4 points)

You want to separate out two cases:

- Cases where the repeat rows for one employer are due to duplicated data 
- Cases where the repeat rows for one employer represent repeated violations for different issues

There are various ways to check duplicates in these data: e.g., converting `Violation` to lowercase or replacing spelled-out states with two-dig state codes.

We're going to use the simple rule of:

- A row is a duplicate if, within an employer (defined by Name + City, State), the Start date for each row's violation is the same 

To begin to check this, reshape `mult_debar` to a wide dataframe (`mult_debar_wide`) with the following columns, treating the `Name` and `City, State` as the index for the pivot:

- Name
- City, State
- start_date_viol1
- start_date_viol2

For a successful reshaping, make sure each row in `mult_debar` shows only a single business (no duplicates under `Name`) and isn't missing either `start_date_viol1` or `start_date_viol2`.

Show the contents of `mult_debar` and its shape.

In [ ]:
## your code here

## 1.5 Filter out duplicates from original debar data (6 points)

A. Using `mult_debar_wide`, add a column `is_dup` that takes value of True for cases where start_date_viol1 == start_date_viol2 marking the row as a duplicate

B. Going back to the original long-format data you loaded at the beginning, `debar`, filter as follows:
- For employers where `is_dup == True` as indicated by your wide-format dataframe, only keep `violnum == viol1`
- For all other employers (so is_dup == False and ones we didnt need to check duplicates for), keep all violnum
- Remove the `is_repeated` column from the `debar` data

**Hint**: you can complete part B without a for loop; `pd.concat` with axis = 0 (row binding) is one way

Call the resulting dataframe `debar_clean` and print the shape and # of unique employer names

In [ ]:
## your code here

# 2. Merging and regex (19 points total)



## 2.1 Load data on job postings (1 point)

The previous dataset contains a small subset of employers who faced temporary bans due to violations of H-2A program regulations. Since most of the bans have expired, let's see which of those employers posted new H-2A jobs in the first quarter of 2021.

First, load the `jobs.csv` data stored in `pset3_inputdata`.

In [ ]:
# your code here to load the data 



The most notable column names mean as follows:
- `CASE_NUMBER`:	Administrative identifier for an employer's H-2A visa application
- `EMPLOYER_NAME`:	Employer name
- `EMPLOYER_CITY`:	Employer city
- `EMPLOYER_STATE`:	Employer state
- `EMPLOYER_ADDRESS_1`:	Employer address (only need to use if doing the geocoding extra credit)

##  2.2 Try inner join on employer name  (2 points)

- Use the `EMPLOYER_NAME` field of the `jobs` dataset
- Use the `Name` field of the `debar_clean` dataset 

A. Use pd.merge with an inner join on those fields to see whether there are any exact matches. 

B. If there are exact matches, print the row(s) with exact matches



In [ ]:
## your code here

## 2.3 Targeted regex (11 points total)

You want to see if you can increase the exact match rate with some basic cleaning of each 
of the employer name fields in each dataset 

### 2.3.1 Convert to upper (3 points)

A. Convert the `EMPLOYER_NAME` and `Name` fields to uppercase using list comprehension rather than df.varname.str.upper() (it's fine to do a separate list comprehension line for each of the two columns)

B. Print a random sample of 15 values of each result

C. Assign the full vector of uppercase names back to the original data, writing over the original `EMPLOYER_NAME` and `Name` columns 


In [ ]:
## insert your code to turn into uppercase here


In [ ]:
## insert your code for the random sample

In [ ]:
## insert your code for assigning the uppercase names back to the data

### 2.3.2 Clean up punctuation (4 points)

You notice that INC, CO, and LLC are sometimes followed by a period (.) but sometimes not

A. For each dataset, write a regex pattern using `re.sub` to remove the . but only if it's preceded by INC, LLC, or CO 

Make sure LLC, INC, CO remain part of the string but just without the dot

B. Test the pattern on the positive and negative example we provide below and print the result. See the Github issue for examples of what to return


**Hint**: https://stackoverflow.com/questions/7191209/python-re-sub-replace-with-matched-content



In [ ]:
pos_example_1 = "CISCO PRODUCE INC."
pos_example_2 = "AVOYELLES HONEY CO., LLC"
neg_example = "E.V. RANCH LLP"

In [ ]:
## insert your code here with the regex pattern for part A

## insert your code to use re.sub to apply the pattern to the test cases for part B

### 2.3.3 Clean employer names (4 points)

Use that pattern in conjunction with `re.sub` and list comprehension to clean the employer name columns in each dataset. Save the new columns as `name_clean` in each. Then, use row subsetting to (1) subset to rows that changed names and (2) for:

- `debar_clean` print the `Name` and `name_clean` columns
- `jobs` print the `EMPLOYER_NAME` and `name_clean` columns

Make sure to use the uppercase versions of the variables


In [ ]:
## your code here to clean the columns

In [ ]:
## your code here to print the head

## 2.4 More joins and more cleaning (5 points)

A. Conduct another inner join between `jobs` and `debar_clean` now using the `name_clean` column; print the result. Did the cleaning result in any more employers matched between the two datasets?

B. Create a new column in `debar_clean` called `name_clean_2` that uses regex to take the following name in that dataset:

- `SLASH E.V. RANCH LLP` in the `debar_clean` dataset

And cleans it up so that it matches with this employer in `jobs`

- `SLASH EV RANCH` in the `jobs` dataset

Eg a pattern to remove the dots in the EV and the space+LLP-- you can apply the pattern to all employer names in debar_clean (so don't need to worry about only applying it to that one employer)


C. Conduct a left join using `name_clean_2` as the join column where the left hand dataframe is `jobs`; right hand dataframe is `debar_clean`, store the result as a dataframe, and print the rows where the merge indicator indicates the row was found in both dataframe. Write a sentence describing how name cleaning affected the match results.

**Note**: this manual cleaning process is inefficient and may miss other likely matches. A better approach would be fuzzy matching, which would recognize that 'Slash EV ranch' is a highly similar string to 'slash ev ranch llp' and match them without us needing to use regex to make the strings identical. We may talk about this in class later if time!

In [ ]:
## your code here

(your interpretation here)

# 3. Regex to separate companies from individuals (6 points)

You notice some employers in `debar_clean` have both the name of the company and the name of individual, e.g.:
    
COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*

Use the uppercase/cleaned `name_clean` in `debar_clean`

A. Write a regex pattern that does the following:
    - Captures the pattern that occurs before COMPANY if (COMPANY) is in string; so in example above, extracts COUNTY FAIR FARM 
    - Captures the pattern that occurs before INDIVIDUAL if (INDIVIDUAL) is also in string -- so in above, extracts ANDREW WILLIAMSON (so omit the "and")
    
B. Test the pattern on `pos_example` and `neg_example`-- make sure former returns a list (if using find.all) or match object (if using re.search) with the company name and individual name separated out; make sure latter returns empty
    
**Hints and resources**: for step A, you can either use re.search, re.match, or re.findall; don't worry about matching B&R Harvesting and Paul Cruz (Individual)

- Same regex resources as above
    

In [ ]:
pos_example = "COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*"
neg_example = "CISCO PRODUCE INC"

## your code here to define the pattern

## your code here to apply it to the pos_example

## your code here to apply it to the negative example

C. Iterate over the `name_clean` column in debar and use regex to create two new columns in `debar_clean`:
   - `co_name`: A column for company (full `name_clean` string if no match; pattern before COMPANY if one extracted)
   - `ind_name`: A column for individual (full `name_clean` string if no match; pattern before INDIVIDUAL if one extracted)
 


In [ ]:
# your code here

   
D. Print three columns for the rows in `debar_clean` containing the negative example and positive example described above (county fair farm and cisco produce):

- `name_clean`
- `co_name`
- `ind_name`
- `Violation`

**Note**: as shown in the outcome there may be duplicates of the same company reflecting different violations

In [ ]:
# your code here

# 4. Optional extra credit: Geospatial visualization (2 points)

Geocode the employer addresses in `jobs` and plot the addresses of jobs as points overlaid on a map of Georgia. This involves Googling and using external sources to figure out the code (a common practice in real-life data science), since we haven't spatial data in the course. 

**Hints:**
- Relevant columns include `EMPLOYER_ADDRESS_1` 
- The geocoding might have a long runtime, so feel free to implement it in a separate `.py` script that you submit alongside your notebook and to just read in the geocoded data

**Resources:**
- [Discussion of geocoding addresses -> lat/long](https://www.natasshaselvaraj.com/a-step-by-step-guide-on-geocoding-in-python/)
- [Discussion of plotting lat/long dots against a map using geopandas](https://towardsdatascience.com/plotting-maps-with-geopandas-428c97295a73)

In [ ]:
## your code here